In [33]:
!pip install allennlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3

# ELMo пример

In [39]:
# Импорт необходимых библиотек и модулей
import pandas as pd  # Библиотека для работы с табличными данными (DataFrames)
import numpy as np   # Библиотека для вычислительных операций
import os            # Модуль для работы с файловой системой
import sys           # Модуль для доступа к некоторым переменным и функциям Python

# Импорт модулей Keras (из tensorflow) для обработки текстовых данных и создания нейронных сетей
from tensorflow.keras.preprocessing.text import Tokenizer     # Для токенизации текста
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Для стандартизации длины последовательностей, Seq2Seq
from tensorflow.keras.utils import to_categorical, plot_model  # Для преобразования категориальных меток и визуализации модели
from tensorflow.keras.layers import Embedding                 # Слой для создания векторных представлений слов
from tensorflow.keras.initializers import Constant          # Для инициализации весов

# Импорт слоев Keras для создания нейронной сети
from tensorflow.keras.layers import Dense, Dropout, Activation, Input  # Основные слои для полносвязной сети
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D    # Слои для конволюционных нейронных сетей
from tensorflow.keras.models import Sequential                    # Класс для создания последовательной нейронной сети
import spacy # обработка русского и английского языка (аналог nltk)

In [8]:
# nlp = spacy.load('en', disable=['parser', 'tagger', 'ner', 'textcat']) # en - для английского язык, старые версии spacy

In [35]:
!pip install spacy
!python -m spacy download en_core_web_sm

2024-09-25 18:55:15.334561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 18:55:15.426135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 18:55:15.438880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 18:55:19.041418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstall

In [36]:
nlp = spacy.load("en_core_web_sm")

In [37]:
MAX_NUM_WORDS = 100000 # максимальное количество слов, которые будут учтены при подготовке данных
MAX_SEQUENCE_LENGTH = 1000 # максимальная длина последовательности слов
EMBEDDING_DIM = 100 # размерность векторов для слов
num_classes = 2 # положительные и отрицательные отзывы

In [38]:
# data: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
train_data_df = pd.read_csv('imdb_train_pandas_datafram.csv')
test_data_df = pd.read_csv('imdb_test_pandas_datafram.csv')

In [40]:
train_data_df.head() # распечатаем датасет IMDB

,sentence,sentiment,polarity
0,This outstanding Argentine independent film is...,10,1
1,This movie is just plain terrible. Poor John S...,2,0
2,"I rented this tape a couple of years ago, and ...",1,0
3,"""Jason Priestly stars as 'Breakfast', a psycho...",3,0
4,Steve Carell stars as a person who you can rel...,7,1


In [41]:
# Токенизация английских слов со spicy
def use_spacy_segmented_words(a_text_sentence):
    doc = nlp(a_text_sentence) # обработка с помощью spacy
    token_list = [token for token in doc]
    return token_list

In [42]:
# Обработать cut_text до фиксированной длины
# обрезаем датасет до фиксированной длины (100000 - слов в отзывах)
def padding_cut_text(datafram_cut_text, MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH):
    raw_cut_text_len = len(datafram_cut_text)
    if raw_cut_text_len >= MAX_SEQUENCE_LENGTH:
        return datafram_cut_text[:MAX_SEQUENCE_LENGTH]
    else:
        datafram_cut_text += ["" for _ in range(MAX_SEQUENCE_LENGTH - raw_cut_text_len)]
        return datafram_cut_text

In [43]:
# Отдельный текст объекта и метки
def slipe_text_label(data_pandas_datafram):
    x_list, label_list = [], []
    # 0, сама строка и т.д.
    for row_index, a_row in data_pandas_datafram.iterrows():
        raw_text = a_row[0]
        raw_cut_text = use_spacy_segmented_words(raw_text)
        raw_cut_padding_text = padding_cut_text(raw_cut_text)
        raw_cut_padding_text = [str(x) for x in raw_cut_padding_text]
        x_list.append(raw_cut_padding_text)
        label_list.append(a_row[1])
    return x_list, label_list

In [ ]:
raw_x_train, raw_y_train = slipe_text_label(train_data_df) # тренировочная, отделяем от метки (разделяем столбцы)

raw_x_test, raw_y_test = slipe_text_label(test_data_df) # тестовая, отделяем от метки (разделяем столбцы)

<ipython-input-43-28186213440b>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  raw_text = a_row[0]
<ipython-input-43-28186213440b>:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_list.append(a_row[1])


In [ ]:
from allennlp.commands.elmo import ElmoEmbedder
# https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5 # модель, веса
# https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway_5.5B/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json # настройки
options_file = "/home/b418/jupyter_workspace/yuanxiao/elmo_data/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json" # настройки elmo
weight_file = "/home/b418/jupyter_workspace/yuanxiao/elmo_data/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5" #сохранная модель с весами

# Предложения, на этапе препроцессинга прогоняем через Elmo, этап индексации
elmo = ElmoEmbedder(options_file, weight_file) # слой Elmo эмбеддингов

In [ ]:
# Эмбеддинги ELMo для всех документов (всех предложений) в нашем наборе данных. Загрузить их
# в матрице numpy, так что вычислить вложения ELMo только один раз.
# сокращение признаков в тексте или слов
def create_elmo_embeddings(elmo, documents, max_sentences = MAX_SEQUENCE_LENGTH):
    num_sentences = min(max_sentences, len(documents)) if max_sentences > 0 else len(documents) # обрезка
    print("\n\n:: Lookup of "+str(num_sentences)+" ELMo representations. This takes a while ::")
    embeddings = []
    documentIdx = 0 # первыц индекс
    for elmo_embedding in elmo.embed_sentences(documents):
        document = documents[documentIdx]
        # Возьмем значение третьего вектора
        third_elmo_embedding = elmo_embedding[2]
        embeddings.append(third_elmo_embedding)
        # Некоторая информация о прогрессе
        documentIdx += 1
        percent = 100.0 * documentIdx / num_sentences
        line = '[{0}{1}]'.format('=' * int(percent / 2), ' ' * (50 - int(percent / 2)))
        status = '\r{0:3.0f}%{1} {2:3d}/{3:3d} sentences'
        sys.stdout.write(status.format(percent, line, documentIdx, num_sentences))

        if max_sentences > 0 and documentIdx >= max_sentences:
            break
    return embeddings


x_train_elmo = create_elmo_embeddings(elmo, raw_x_train[:100], 100) # сокращаем признаки
x_text_elmo = create_elmo_embeddings(elmo, raw_x_test[:100], 100) # сокращаем признаки

In [ ]:
len(x_text_elmo) # проверяем, что обрезалось

In [ ]:
x_train_elmo_1 = np.array(x_train_elmo)

In [ ]:
x_text_elmo_1 = np.array(x_text_elmo)

## Сократим

In [ ]:
y_train = np.array(raw_y_train[:100]) # y_train - классы тренировочных отзывов
y_test = np.array(raw_y_test[:100]) # y_test - классы тестовых отзывов

## Первая модель

In [ ]:
def creat_elmo_v1_model():
    model = Sequential() # последовательная модель Seq2Seq
    # Conv1D (1D-конволюция), 250 нейрон-фильтр, 3 элемента преобразуется фильтром, 'same' - заполнение вектора
    # MAX_SEQUENCE_LENGTH - максимальная последовательность, 1024 - размерность вектора ELMo
    model.add(Conv1D(filters=250, kernel_size=3, padding='same', input_shape=(MAX_SEQUENCE_LENGTH,1024)))
    model.add(GlobalMaxPooling1D()) # глобальное максимальное свертывание 1D, вектор максимальных значений
    model.add(Dense(1)) # выход 1 нейрон
    model.add(Activation('sigmoid'))

    plot_model(model,
               to_file="IMDB_ELMo_Preprocessing.png",
               show_shapes=True)
    model.compile(
              loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
    return model

In [ ]:
creat_elmo_v1_model = creat_elmo_v1_model() # Создаем модель ELMo

## Обучающая модель

In [ ]:
%%time
creat_elmo_v1_model.fit(x_train_elmo_1, y_train,
          batch_size=2, # стандартных batch 32, 64... для них достаточно 16, как тренируется на 2
          epochs=10, # в течении 10 эпох, для тренировочной выборки достаточно 3-4 эпох, для тестовой недостаточно
          validation_split=0.2) # 20% данных для валидации

## Тестовая модель

In [ ]:
creat_elmo_v1_model.evaluate(x_text_elmo_1, y_test)

# ELMo как слой последовательной сети


In [2]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
Y

y
  Successfully uninstalled tensorflow-2.17.0


In [4]:
!pip install tensorflow

  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)


In [5]:
# Библиотеки
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from tensorflow.keras import backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
import numpy as np

In [6]:
# Загрузите все файлы из каталога во фрейм данных.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# Объединить положительные и отрицательные примеры, добавить столбец полярности и перемешать.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos")) # положительный
    neg_df = load_directory_data(os.path.join(directory, "neg")) # отрицательный
    pos_df["polarity"] = 1 # положительный
    neg_df["polarity"] = 0 # отрицательный
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Загрузить и обработать файлы набора данных.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz",
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
      extract=True)

    train_df = load_dataset(os.path.join(os.path.dirname(dataset),
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset),
                                      "aclImdb", "test"))

    return train_df, test_df

train_df, test_df = download_and_load_datasets()
train_df.head()

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


,sentence,sentiment,polarity
0,This outstanding Argentine independent film is...,10,1
1,This movie is just plain terrible. Poor John S...,2,0
2,"I rented this tape a couple of years ago, and ...",1,0
3,"""Jason Priestly stars as 'Breakfast', a psycho...",3,0
4,Steve Carell stars as a person who you can rel...,7,1


In [7]:
train_df.to_csv("imdb_train_pandas_datafram.csv", encoding='utf-8-sig', index=False)
test_df.to_csv("imdb_test_pandas_datafram.csv", encoding='utf-8-sig', index=False)

In [8]:
train_df.head(30)

,sentence,sentiment,polarity
0,This outstanding Argentine independent film is...,10,1
1,This movie is just plain terrible. Poor John S...,2,0
2,"I rented this tape a couple of years ago, and ...",1,0
3,"""Jason Priestly stars as 'Breakfast', a psycho...",3,0
4,Steve Carell stars as a person who you can rel...,7,1
5,Tim (Gary Daniels) wants desperately to break ...,2,0
6,"I have read the short story by Norman Maclean,...",10,1
7,This movie starts presenting a somehow origina...,2,0
8,"This is a racist movie, but worthy of study an...",7,1
9,"I never much liked the Myra movie, tho I appre...",3,0


In [26]:
%%time
# Теперь экземпляр модели elmo
# url-адрес модели elmo https://tfhub.dev/google/elmo/2
elmo_model = hub.load("https://tfhub.dev/google/elmo/2")
elmo_model.signatures['default']

CPU times: user 2.01 s, sys: 137 ms, total: 2.15 s
Wall time: 2.16 s


<ConcreteFunction () -> Dict[['elmo', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['default', TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None)], ['lstm_outputs1', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['word_emb', TensorSpec(shape=(None, None, 512), dtype=tf.float32, name=None)], ['sequence_len', TensorSpec(shape=(None,), dtype=tf.int32, name=None)], ['lstm_outputs2', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)]] at 0x7AFF0CFBF880>

In [27]:
# Модель

# Создаем функцию для интеграции модели tensorflow с моделью Keras
# Для этого требуется явное приведение тензора к строке из-за Keras
# tf.squeeze(tf.cast(x, tf.string)) - Удаляет лишние измерения, если они есть.
# tf.cast(x, tf.string) - преобразует в строки
# signature="default" - сигнатура по умолчанию
# as_dict=True - на выходе словарь
def ElmoEmbedding(x):
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [28]:
input_text = layers.Input(shape=(1,), dtype=tf.string) # dtype=tf.string строка, shape=(1,) размерности 1
embedding = layers.Lambda(ElmoEmbedding, output_shape=(1024,))(input_text) # Предложения, на этапе обучения модели прогоняем через Elmo, этап работы с классификацией
dense = layers.Dense(256, activation='relu')(embedding) # 256 нейронов
pred = layers.Dense(1, activation='sigmoid')(dense) # выходной слой, слой предсказания класса 1 - положит отзыв, 0 - отрицательный

model = Model(inputs=[input_text], outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary() # краткое описание архитектуры модели

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_4 (Lambda)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 262,657 (1.00 MB)

 Trainable params: 262,657 (1.00 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Создать наборы данных (для запоминания требуется всего до 150 слов)
# тренировка
train_text = train_df['sentence'].tolist() # отзывы
train_text = [' '.join(t.split()[0:150]) for t in train_text] # обрезаем до 150 слов
train_text = np.array(train_text, dtype=object)[:, np.newaxis] # преобразовываем в numpy массив
train_label = train_df['polarity'].to_numpy() # классы отзыва, 1, 0

# тест
test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['polarity'].to_numpy()

In [31]:
%%time
# Дотренировка на новых данных
model.fit(train_text, # тренировочная выборка
          train_label, # классы
          epochs=5,
          batch_size=256, # 32, 64, 128, 256, 512
          validation_split=0.2) # 20% валидации для проверки


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


## Оценочная модель

In [32]:
%%time
model.evaluate(test_text, test_label)